# Описание задания

В рамках данного задания, студент должен создать и оценить 4 типа рекомендательных систем:
* Content-based RS

Каждая рекомендательная система - отдельное подзадание. Подзадание считается выполненным, если студент создал рекомендательную систему, которая **лучше (или хуже, но не более чем на 10%)** системы, созданной автором данного блокнота. Системы оцениваются с использованием метрики ``HR@N``, описанной ниже.

# Детальное описание

#### Данные: 
Датасет представлен множеством отзывов к компьютерным играм (объектам) от пользователей Amazon. Каждый отзыв представлен в виде JSON-структуры со следующими полями:
* идентификатор пользователя - reviewerID
* идентификатор объекта - asin
* текст отзыва - reviewText
* рейтинг - overall
* время публикации обзора - unixReviewTime
* другие поля, не использованные автором этого блокнота (смотри полное описание JSON [тут](http://jmcauley.ucsd.edu/data/amazon/))

У каждого объекта есть как минимум 5 отзывов, каждый пользователь написал как минимум 5 отзывов. 
#### Цель: 
Построить рекомендательную систему, предсказывающую объекты, которые пользователь приобретет в ближайшем будущем. Для упрощения мы считаем, что пользователь приобрел объект, если он написал про него отзыв.
#### Подготовка данных:
Данные разделены на тренировочную и тестовую выборки по времени публикации отзывов. Первые 80% данных (более старые) используются как тренировочная выборка, остальные - как тестовая. 

Построение рекомендательной системы (т.е., выбор и тренировка моделей, оптимизация параметров и т.д.) осуществляется **только** с использованием тренировочной выборки. Все параметры, использованные в моделях, **должны быть** получены или объяснены с помощью тренировочных данных. Студент вправе использовать тренировочную выборку как его душе угодно. 

Тестирующая выборка используется **только** для оценки рекомендательной системы.

Для построения рекомендательных моделей также можно использовать JSON-поля из датасета, неиспользованные автором этого блокнота.
#### Оценка качества рекомендательной системы
Цель рекомендательной системы - посоветовать пользователю объекты, которые он захочет приобрести. Для оценки качества такой системы мы воспользуемся метрикой `hit-ratio (HR)`. 

$$
HR = \frac{1}{|U_T|}\sum_{u \in U_T} \mathrm{I}(Rel_u \cap Rec_u)
$$

* $U_T$ - множество пользователей из тестовой выборки
* $Rec_u$ - множество объектов, рекомендованных пользователю $u$ 
* $Rel_u$ - множество объектов, оцененных пользователем $u$ в тестовой выборке
* $\mathrm{I}(Rel_u \cap Rec_u)$ - бинарная функция-индикатор. Функция возвращает 1 если $Rel_u \cap Rec_u \ne \emptyset$, иначе 0

$HR=1$ если для каждого пользователя мы рекомендовали хотя бы один релевантный объект. Так как обычно пользователи просматривают только первые $N$ рекомендаций, мы будем считать метрику $HR@N$, где $N=10$ (т.е. множество $Rec_u$ будет содержать только 10 объектов). 

# Условные обозначения
* `uid` - идентификатор пользователя
* `iid` - идентификатор объекта

# Games RSs

In [1]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
JSON_DATA_PATH = "data/Video_Games_5.json"

## Анализ данных

In [3]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

In [4]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


In [5]:
# проверяем, есть ли случаи, когда один и тот же пользователь оставляет отзывы на один и тот же объект
df.groupby(["uid", "iid"]).review.count().unique()  # ура, таких случаев нет

array([1], dtype=int64)

In [6]:
print("Количество объектов:", df.iid.unique().size)
print("Количество пользователей:", df.uid.unique().size)

Количество объектов: 10672
Количество пользователей: 24303


## Готовим выборки

In [7]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [8]:
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "data/Video_Games_5.json"
N = 10

In [9]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


In [10]:
def clean_df(df, min_review_per_uid, min_review_per_iid):
    """Функция удаляет из df строки, соответствующие пользователям и объектам, 
    у которых меньше min_review_per_uid и min_review_per_iid отзывов соответственно
    """
    _df = df.copy()
    while True:
        review_per_uid = _df.groupby("uid").review.count()
        bad_uids = review_per_uid[review_per_uid < min_review_per_uid].index
    
        review_per_iid = _df.groupby("iid").review.count()
        bad_iids = review_per_iid[review_per_iid < min_review_per_iid].index
        
        if bad_uids.shape[0] > 0 or bad_iids.shape[0] > 0:
            _df = _df[(~_df.uid.isin(bad_uids)) & (~_df.iid.isin(bad_iids))]
        else:
            break
    return _df

 ## Метрика

Для упрощения тестирования предлагается использовать словарь следующего типа:

```python
recs = {
    uid_1: {
        iid_1: score_11,
        iid_2: score_12,
        ...
    },
    uid_2: {
        iid_1: score_21,
        iid_2: score_22,
        ...
    },
    ...
}
```

где `uid_i` - идентификатор тестового пользователя, `iid_j` - идентификатор рекомендованного объекта, а `score_ij` - предсказанный рейтинг/вес объекта `j` для пользователя `i`.

In [11]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [12]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## Content-based RS

Простая content-based рекомендательная система описывает пользователей и объекты как вектора в некотором N-мерном пространстве фич. Вектор объекта показывает, насколько объект принадлежит к той или иной фиче. Вектор пользователя показывает, насколько пользователь предпочитает ту или иную фичу. Рекомендации строятся путем поиска объектов, чьи вектора похожи на вектор предпочтений пользователя. Предполагается, что чем более похожи вектора пользователя и объекта, тем интереснее этот объект пользователю.

#### `HR@10` для content-based, модели созданной автором блокнота: 0.065

### Сделаем предобработку rewiew

In [13]:
# соберем данные для обучения TfidfVectorizer
texts = [m for m in training_df['review']]
len(texts)

185427

In [14]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf = TfidfVectorizer(stop_words='english', analyzer='word', tokenizer=None, max_features=25000 )
tfidf = TfidfVectorizer(stop_words='english', analyzer='word', tokenizer=None)
tfidf.fit(texts)

Wall time: 44.2 s


In [15]:
result = sorted(tfidf.vocabulary_.items(), key=lambda x:x[1], reverse=True)[:20]
result

[('zzzzzzzzzzzzzzzzzzzz', 197238),
 ('zzzzzzzzzzzzzzzzzzz', 197237),
 ('zzzzzzzzzzzzzz', 197236),
 ('zzzzzzzzzzzziiiiiiiiiinnnnnnnnnnngggggggggg', 197235),
 ('zzzzzzzzzzzz', 197234),
 ('zzzzzzzzzzz', 197233),
 ('zzzzzzzzzz', 197232),
 ('zzzzzzzzz', 197231),
 ('zzzzzzzz', 197230),
 ('zzzzzzz', 197229),
 ('zzzzzz', 197228),
 ('zzzzzpt', 197227),
 ('zzzzzp', 197226),
 ('zzzzz', 197225),
 ('zzzzt', 197224),
 ('zzzzs', 197223),
 ('zzzz', 197222),
 ('zzz', 197221),
 ('zzkchkchkchkkchckrkchhhhhhhhhkkkkkak', 197220),
 ('zzes', 197219)]

## Посчитаем TFIDF

In [17]:
# сделаем матрицу из обьектов и их описаний
training_df['review']='  '+training_df['review']
#Object=training_df.groupby('iid')['review'].sum()
#len(Object)

In [18]:
# попробуем как Турал
review_ftr_m = tfidf.transform(training_df.review)
#сразу умножаем на рейтинги
review_ftr_m = review_ftr_m.multiply(training_df.rating.values.reshape(-1, 1)).tocsr()

In [19]:
# маритцу получили, по хорошему ее бы не мешало почистить и обработать
# как это делать пока не очень понятно
from sklearn.preprocessing import normalize
from scipy.sparse import vstack

# возьмем функцию Турала, она быстрее
def _prepare_iid_data(df, review_ftr_m): 
    iid_to_row = {}
    rows = []
    # не самый оптимальный group by       
    for row_id, iid in enumerate(df.iid.unique()):
        iid_to_row[iid] = row_id # свяжем
        
        # вариант ищем iddtf * рейтинг и потом складываем вектора
        iid_ftr_m = csr_matrix(review_ftr_m[np.where(df.iid == iid)[0]].sum(axis=0))
        rows.append(iid_ftr_m)
    
    iid_ftr_m = normalize(vstack(rows, format='csr'))
    return iid_to_row, iid_ftr_m


In [20]:
# маритцу получили, по хорошему ее бы не мешало почистить и обработать
# как это делать пока не очень понятно
from sklearn.preprocessing import normalize
from scipy.sparse import vstack

In [21]:
%%time
iid_to_row, iid_profile = _prepare_iid_data(training_df, review_ftr_m)
len(iid_to_row)

Wall time: 58.2 s


In [22]:
# получили большую матрицу обьектов
iid_profile

<10098x197239 sparse matrix of type '<class 'numpy.float64'>'
	with 7356087 stored elements in Compressed Sparse Row format>

## Создадим профиль пользователя

### Поработаем с рейтингом пользователя

In [23]:
# сделаем пока просто подсчитаем средний рейтинг который пользователь ставит и если текущий быше нравится
# можно примеить специальные функции для рейтига /// если хватит времени
#mean_tr=training_df.groupby('uid')['rating'].mean().to_frame()
#training_df = pd.merge(training_df, mean_tr, how='left', left_on='uid', right_index=True )

In [24]:
# народ настроен  более положительно чем отрицательно, это хорошо
# точно нужно применять другую функцию для оценки рейтинга и возможно поробовать сделать нравится/не нравится/нормально
#training_df["liked"] = (training_df.rating_x >= training_df.rating_y).astype(int)
#training_df.liked.value_counts()

In [25]:
training_df.rename(columns={'rating_x' : 'rating'}, inplace=True)
#training_df.columns.values[3] = 'rating'
training_df.query('uid==5')

,uid,iid,review,rating,dt
5,5,0,"Overall this is a well done racing game, w...",4.0,1368230400
93459,5,5000,My family loves this game. We all play to...,5.0,1301788800
121923,5,6308,Need for Speed - Shift is a great racing g...,4.0,1346630400
123196,5,6349,Overall this is a well done racing racing ...,4.0,1368230400
125474,5,6428,I have read the many negative reviews. I ...,5.0,1264809600
168532,5,8069,I find Need for Speed Shift 2 a very fun g...,5.0,1309824000
214274,5,9935,Graphics are excellent. Would be nice to ...,5.0,1360454400


In [26]:
temp_crs=csr_matrix(tfidf.transform(training_df[training_df.uid == 5].review)).transpose()
temp_crs=temp_crs.multiply(training_df[training_df.uid == 5].rating)
temp_crs=csr_matrix(temp_crs.sum(axis=1))

temp_crs

#*training_df[training_df.uid == 5].rating

<197239x1 sparse matrix of type '<class 'numpy.float64'>'
	with 218 stored elements in Compressed Sparse Row format>

In [27]:
# Моя версия
def _prepare_uid_data2(df, tfidf):  
        uid_to_row = {}
        rows = []
        
        # вытаскиваем все существующие отзывы поьзователя
        for row_id, uid in enumerate(df.uid.unique()):
            uid_to_row[uid] = row_id # свяжем
            
            # вариант ищем iddtf * рейтинг и потом складываем вектора
            uid_ftr_m=csr_matrix(tfidf.transform(training_df[training_df.uid == uid].review))#.transpose()
            #uid_ftr_m=uid_ftr_m.multiply(training_df[training_df.uid == uid].rating).transpose()
            uid_ftr_m=csr_matrix(uid_ftr_m.sum(axis=0))
            
            rows.append(uid_ftr_m)
            
        uid_ftr_m = normalize(vstack(rows, format='csr'))
        return uid_to_row, uid_ftr_m
            
            

In [28]:
%%time
# постоим матрицу пользователей
#uid_to_row, uid_profile = _prepare_uid_data2(training_df, tfidf)
#len(uid_to_row)

Wall time: 0 ns


In [29]:
# версия Турала
def _prepare_uid_data(df, iid_to_row, iid_ftr_m):  
        uid_to_row = {}
        rows = []
        
        # gr_df - кусок df с данными одного пользователя 
        for gr_id, gr_df in df.groupby("uid"):
            uid = gr_df.uid.values[0]
            
            # поиск объектов и пользовательских рейтингов для них
            iid_rows = []
            ratings = []
            for iid, rating in zip(gr_df.iid.values, gr_df.rating.values):
                if iid in iid_to_row:
                    iid_rows.append(iid_to_row[iid])
                    ratings.append(rating)
                  
            # создание профиля пользователя
            if iid_rows:
                ratings = np.array(ratings).reshape(-1, 1)
                uid_ftr_m = csr_matrix(iid_ftr_m[iid_rows].multiply(ratings).sum(axis=0)) 
                uid_to_row[uid] = len(uid_to_row)
                rows.append(uid_ftr_m)
            
        uid_ftr_m = normalize(vstack(rows, format='csr'))
        return uid_to_row, uid_ftr_m

In [30]:
%%time
uid_to_row, uid_profile = _prepare_uid_data(training_df, iid_to_row, iid_profile)

Wall time: 2min 35s


In [31]:
uid_profile

<22215x197239 sparse matrix of type '<class 'numpy.float64'>'
	with 149385302 stored elements in Compressed Sparse Row format>

In [32]:
# немного упрощаем себе жизнь (для подсчета рекомендаций с использованием косинусной меры)
ftr_iid_m = iid_profile.T.tocsr()
row_to_iid = {row_id: iid for iid, row_id in iid_to_row.items()}

In [33]:
u_row_id = uid_to_row[256]
print(u_row_id)
u_row = uid_profile[u_row_id]

u_recs = u_row.dot(ftr_iid_m)
np.sort(u_recs.data, axis=None)[-10:]

234


array([ 0.64544241,  0.64919771,  0.65139977,  0.65471729,  0.66222475,
        0.67183736,  0.68208319,  0.68493811,  0.72738134,  0.7500904 ])

In [34]:
from scipy.stats.stats import pearsonr

In [35]:
def get_recs(uid, top):
    recs = {}
    if uid in uid_to_row:
        u_row_id = uid_to_row[uid]
        u_row = uid_profile[u_row_id]
        
        # самописный cosine similarity
        u_recs = u_row.dot(ftr_iid_m)

        for arg_id in np.argsort(u_recs.data)[-top:]:
            row_id = u_recs.indices[arg_id]
            score = u_recs.data[arg_id]
            recs[row_to_iid[row_id]] = score
    return recs


In [38]:
    def get_batch_recs( uids, top):
        return {uid: get_recs(uid, top) for uid in uids}

In [39]:
%%time
rec_dict=get_batch_recs(test_df.uid,10)

Wall time: 6min 26s


In [40]:
# тупо берем первые 10 по суммарным баллам за просмотр
hit_ratio(rec_dict,test_dict)

0.06500366837857667

## Эксперименты и выводы

####  0-0 Результат по умолчанию (все фичи, TFIDF юзера по SUM(РЕВЬЮ*RATING), No Feature Selection). Результат теста hit-ratio (HR): 0.049

####  0-1 Результат по умолчанию (все фичи, TFIDF юзера по SUM(РЕВЬЮ), No Feature Selection). Убрал перемножение на рейтинг. Результат теста hit-ratio (HR):  0.0491562729273661

####  0-2 Вариант Турала (все фичи,  uid_profile по его объектам из iid_profile SUM(РЕВЬЮ*RATING), No Feature Selection).  Результат теста hit-ratio (HR): 0.06544387380777696

####  0-3 Вариант Турала (все фичи, uid_profile по его объектам из iid_profile SUM(РЕВЬЮ), No Feature Selection).  Результат теста hit-ratio (HR):  0.06206896551724138  - как и предполагалось ухудшило результат

####  0-4 Вариант Турала (все фичи, iid_profile SUM(РЕВЬЮ), uid_profile по его объектам из iid_profile SUM(РЕВЬЮ*RATING), No Feature Selection).  Результат теста hit-ratio (HR):  0.06544387380777696 - точно такой же как и у iid_profile * RATING, наверное веса оценок iid при косинусной мере, не столь важны а вот для матирцы uid они позволяют более точно определить предпочтения пользователя 

####  1-1 Вариант Турала ( TFIDF , uid_profile по его объектам из iid_profile SUM(РЕВЬЮ), No Feature Selection).  Результат теста TFIDF max_features=100 000  hit-ratio (HR):  0.0632428466617755


####  1-1 Вариант Турала ( TFIDF , uid_profile по его объектам из iid_profile SUM(РЕВЬЮ), No Feature Selection).  Результат теста 
TFIDF max_features=100 000   hit-ratio (HR):  0.0632428466617755   
50 0000  (HR): 0.06456346294937637  
25 000 (HR): 0.06471019809244315
10 000 (HR): 0.0632428466617755